<a href="https://colab.research.google.com/github/GreatLearningAIML1/delhi-may19-batch-sharma-chandan/blob/master/Statistical_NLP_R8_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Description**

Classification is probably the most popular task that you would deal with in real life.Text in the form of blogs, posts, articles, etc. is written every second. It is a challenge to predict the information about the writer without knowing about him/her. We are going to create a classifier that predicts multiple features of the author of a given text.We have designed it as a Multilabel classification problem.

**Dataset**

Blog Authorship Corpus
Over 600,000 posts from more than 19 thousand bloggers
The Blog Authorship Corpus consists of the collected posts of 19,320 bloggers gathered from blogger.com in August 2004. The corpus incorporates a total of 681,288 posts and over 140 million words - or approximately 35 posts and 7250 words per person. Each blog is presented as a separate file, the name of which indicates a blogger id# and the blogger’s self-provided gender, age, industry, and astrological sign. (All are labeled for gender and age but for many, industry and/or sign is marked as unknown.)

All bloggers included in the corpus fall into one of three age groups:

8240 "10s" blogs (ages 13-17),

8086 "20s" blogs(ages 23-27)

2994 "30s" blogs (ages 33-47)

For each age group, there is an equal number of male and female bloggers.
Each blog in the corpus includes at least 200 occurrences of common English words. All formatting has been stripped with two exceptions. Individual posts within a single blogger are separated by the date of the following post and links within a post are denoted by the label urllink.

Link to dataset: https://www.kaggle.com/rtatman/blog-authorship-corpus

In [1]:
# imports
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score, recall_score, precision_score
from nltk.stem import WordNetLemmatizer

1. Load the dataset (5 points)
a. Tip: As the dataset is large, use fewer rows. Check what is working well on your
machine and decide accordingly.

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/stat_nlp/blogtext.csv .

In [38]:
# Read data 
corpus_df_sample = pd.read_csv("blogtext.csv",nrows=5000)
corpus_df_sample.head(10)


,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...


2. Preprocess rows of the “text” column (7.5 points)

    a. Remove unwanted characters

    b. Convert text to lowercase

    c. Remove unwanted spaces

    d. Remove stopwords


In [39]:
#Removing unwanted / special characters
corpus_df_sample['text'] = corpus_df_sample['text'].str.replace('[^A-Za-z]',' ')
corpus_df_sample["text"].loc[0]

'           Info has been found          pages  and     MB of  pdf files  Now i have to wait untill our team leader has processed it and learns html          '

In [40]:
# Coverting to lower case
corpus_df_sample['text'] = corpus_df_sample['text'].str.lower()
corpus_df_sample["text"].loc[0]

'           info has been found          pages  and     mb of  pdf files  now i have to wait untill our team leader has processed it and learns html          '

In [41]:
#Removing spaces
corpus_df_sample["text"] = corpus_df_sample["text"].str.strip()
corpus_df_sample["text"].loc[0]

'info has been found          pages  and     mb of  pdf files  now i have to wait untill our team leader has processed it and learns html'

In [42]:
corpus_df_sample["text"] = corpus_df_sample["text"].str.split()  # splitting each row of text data into individual words.
# So it can be iterated through to remove only stopwords in next steps.

In [43]:
#remove stop words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
# set of stop words
stop = set(stopwords.words('english'))

def removestopwords(y):   # Function definition
 stopwordremoved = [w for w in y if w not in stop]
 return(" ".join(stopwordremoved))

In [45]:
text_column_size = corpus_df_sample["text"].size
print("text column size :", text_column_size)

# Initialize an empty list to hold the text after stop word removal
cleaner_corpus_df_sample_text = []

# Loop over each text
for i in range( 0, text_column_size):
    cleaner_corpus_df_sample_text.append(removestopwords(corpus_df_sample["text"][i]))

text column size : 5000


In [46]:
cleaner_corpus_df_sample_text[10]

'ah korean language looks difficult first figure read hanguel korea surprisingly easy learn alphabet characters seems easy vocabulary starts oh backwards us sentence structure yikes luckily many options us slow witted foreigners take language course could list urllink joongang article says lot resources urllink well guy motivation jeon ji hyun latest something actually star movies cfs hear means commercial feature positive saw latest movie sunday night hard describe name english version windstruck korean version yeochinso short ne yeojachingu rul sogayhamnida like introduce girlfriend surprisingly titles make sense like website korean english looks quite good actually urllink movie shown theatres subtitles special times info urllink list many theatres seoul click urllink urllink great reason learn korean already married went foreigners well local korean national course korean take picture put urllink movie hof bar update bud mine passed urllink link giordano ad apparently aired korea n

In [47]:
#Replace text column with cleaner_corpus_df_sample_text 
corpus_df_sample["text"] = cleaner_corpus_df_sample_text
corpus_df_sample["text"][10]

'ah korean language looks difficult first figure read hanguel korea surprisingly easy learn alphabet characters seems easy vocabulary starts oh backwards us sentence structure yikes luckily many options us slow witted foreigners take language course could list urllink joongang article says lot resources urllink well guy motivation jeon ji hyun latest something actually star movies cfs hear means commercial feature positive saw latest movie sunday night hard describe name english version windstruck korean version yeochinso short ne yeojachingu rul sogayhamnida like introduce girlfriend surprisingly titles make sense like website korean english looks quite good actually urllink movie shown theatres subtitles special times info urllink list many theatres seoul click urllink urllink great reason learn korean already married went foreigners well local korean national course korean take picture put urllink movie hof bar update bud mine passed urllink link giordano ad apparently aired korea n

In [48]:
#Lemmatization
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    lemm = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return(" ".join(lemm)) 

corpus_df_sample["text"] = corpus_df_sample.text.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
corpus_df_sample.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",info found page mb pdf file wait untill team l...
1,2059027,male,15,Student,Leo,"13,May,2004",team member drewes van der laag urllink mail r...
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture url popups mean s...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",interesting conversation dad morning talking k...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",somehow coca cola way summing thing well early...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",anything korea country extreme everything seem...
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",take read news article urllink joongang ilbo n...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",surf english news site lot looking tidbit kore...


3. As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence (7.5 points)

    a. Label columns to merge: “gender”, “age”, “topic”, “sign”
    
    b. After completing the previous step, there should be only two columns in your dataframe i.e. “text” and “labels” as shown in the below image

In [50]:
#name of available columns 
corpus_df_sample.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [51]:
corpus_df_sample.shape

(5000, 7)

In [52]:
corpus_df_sample.head(2)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",info found page mb pdf file wait untill team l...
1,2059027,male,15,Student,Leo,"13,May,2004",team member drewes van der laag urllink mail r...


In [53]:
# merge 'gender', 'age', 'topic', 'sign'
corpus_df_sample['age'] = corpus_df_sample['age'].astype(str)
corpus_df_sample['labels'] = corpus_df_sample[['gender','age','topic','sign']].apply(lambda x: ','.join(x), axis = 1) 
corpus_df_sample_merged = corpus_df_sample.drop(labels = ['date','gender', 'age','topic','sign','id'], axis = 1)
corpus_df_sample_merged.head()

,text,labels
0,info found page mb pdf file wait untill team l...,"male,15,Student,Leo"
1,team member drewes van der laag urllink mail r...,"male,15,Student,Leo"
2,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoo toolbar capture url popups mean s...,"male,33,InvestmentBanking,Aquarius"


In [54]:
corpus_df_sample_merged.shape

(5000, 2)

4. Separate features and labels, and split the data into training and testing(5 points)

In [55]:

feature = corpus_df_sample_merged['text']
corpus_df_sample_merged['labels'] = corpus_df_sample_merged['labels'].str.lower()
labels = corpus_df_sample_merged['labels']
X_train, X_test, Y_train, Y_test = train_test_split(feature,labels, test_size = 0.33, random_state = 143)
print(Y_train.shape)
print(X_train.shape)
print(Y_test.shape)
print(X_test.shape)

(3350,)
(3350,)
(1650,)
(1650,)


### 5. Vectorize the features (5 points)

    a. Create a Bag of Words using count vectorizer

        i. Use ngram_range=(1, 2)

        ii. Vectorize training and testing features

    b. Print the term-document matrix

In [56]:
# Creating Bag of words
vectorizer = CountVectorizer(min_df = 2,ngram_range = (1,2),stop_words = "english")
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print("X_train shape & sample",X_train.shape)
X_train[0]

X_train shape & sample (3350, 27104)


<1x27104 sparse matrix of type '<class 'numpy.int64'>'
	with 68 stored elements in Compressed Sparse Row format>

### 6. Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label. (5 points)

In [57]:
vectorizer_labels = CountVectorizer(min_df = 1,ngram_range = (1,1),stop_words = "english")
labels_vector = vectorizer_labels.fit_transform(labels)
vectorizer_labels.vocabulary_

{'14': 0,
 '15': 1,
 '16': 2,
 '17': 3,
 '23': 4,
 '24': 5,
 '25': 6,
 '26': 7,
 '27': 8,
 '33': 9,
 '34': 10,
 '35': 11,
 '36': 12,
 '37': 13,
 '39': 14,
 '41': 15,
 '42': 16,
 '44': 17,
 '45': 18,
 '46': 19,
 'accounting': 20,
 'aquarius': 21,
 'aries': 22,
 'arts': 23,
 'automotive': 24,
 'banking': 25,
 'businessservices': 26,
 'cancer': 27,
 'capricorn': 28,
 'communications': 29,
 'consulting': 30,
 'education': 31,
 'engineering': 32,
 'female': 33,
 'gemini': 34,
 'indunk': 35,
 'internet': 36,
 'investmentbanking': 37,
 'law': 38,
 'leo': 39,
 'libra': 40,
 'libraries': 41,
 'male': 42,
 'media': 43,
 'museums': 44,
 'non': 45,
 'pisces': 46,
 'profit': 47,
 'recreation': 48,
 'religion': 49,
 'sagittarius': 50,
 'science': 51,
 'scorpio': 52,
 'sports': 53,
 'student': 54,
 'taurus': 55,
 'technology': 56,
 'virgo': 57}

In [58]:
# Extracing only key value from above dictionary, which contains unique labels. These set of labels will be used as classes in 
# multilabelbinariser further.
label_classes = []  
for key in vectorizer_labels.vocabulary_.keys():
    label_classes.append(key)
    
print(sorted(label_classes))

['14', '15', '16', '17', '23', '24', '25', '26', '27', '33', '34', '35', '36', '37', '39', '41', '42', '44', '45', '46', 'accounting', 'aquarius', 'aries', 'arts', 'automotive', 'banking', 'businessservices', 'cancer', 'capricorn', 'communications', 'consulting', 'education', 'engineering', 'female', 'gemini', 'indunk', 'internet', 'investmentbanking', 'law', 'leo', 'libra', 'libraries', 'male', 'media', 'museums', 'non', 'pisces', 'profit', 'recreation', 'religion', 'sagittarius', 'science', 'scorpio', 'sports', 'student', 'taurus', 'technology', 'virgo']


### 7. Transform the labels - (7.5 points)
As we have noticed before, in this task each example can have multiple tags. To deal with
such kind of prediction, we need to transform labels in a binary form and the prediction will be
a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn

a. Convert your train and test labels using MultiLabelBinarizer

In [59]:
mlb = MultiLabelBinarizer(classes = label_classes)  # initialising multilabelbinariser with all unique possible classes

In [60]:
# Converting entire seT of labels into format required by mlb
labels = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in labels]]
labels[30]

['male', '33', 'investmentbanking', 'aquarius']

In [61]:
labels_trans = mlb.fit(labels) # transforming entire set of lables
labels_trans

MultiLabelBinarizer(classes=['male', '15', 'student', 'leo', '33',
                             'investmentbanking', 'aquarius', 'female', '14',
                             'indunk', 'aries', '25', 'capricorn', '17',
                             'gemini', '23', 'non', 'profit', 'cancer',
                             'banking', '37', 'sagittarius', '26', '24',
                             'scorpio', '27', 'education', '45', 'engineering',
                             'libra', ...],
                    sparse_output=False)

In [62]:
#Convert Y_train into a format as required by mlb 
Y_train = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y_train]]
Y_train[30]

['male', '35', 'technology', 'aries']

In [63]:
Y_train_trans = mlb.transform(Y_train) # transforming Train lables using mlb which is trained on all possible unnique labels on entire data set
Y_train_trans[30]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['communicationsmedia', 'museumslibraries', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [64]:
Y_train_trans.shape

(3350, 58)

In [65]:
#Convert Y_test into a format as required by mlb 
Y_test = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y_test]]
Y_test_trans = mlb.transform(Y_test) # transforming test labels.
print(Y_test[30])

['female', '16', 'indunk', 'leo']


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['communicationsmedia', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


In [66]:
Y_test_trans[30]

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [67]:
len(mlb.classes_)

58

In [68]:
mlb.classes_

array(['male', '15', 'student', 'leo', '33', 'investmentbanking',
       'aquarius', 'female', '14', 'indunk', 'aries', '25', 'capricorn',
       '17', 'gemini', '23', 'non', 'profit', 'cancer', 'banking', '37',
       'sagittarius', '26', '24', 'scorpio', '27', 'education', '45',
       'engineering', 'libra', 'science', '34', '41', 'communications',
       'media', 'businessservices', 'sports', 'recreation', 'virgo',
       'taurus', 'arts', 'pisces', '44', '16', 'internet', 'museums',
       'libraries', 'accounting', '39', '35', 'technology', '36', 'law',
       '46', 'consulting', 'automotive', '42', 'religion'], dtype=object)

### 8. Choose a classifier - (5 points)
In this task, we suggest using the One-vs-Rest approach, which is implemented in
OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a
basic classifier, use LogisticRegression . It is one of the simplest methods, but often it
performs good enough in text classification tasks. It might take some time because the
number of classifiers to train is large.

    a. Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label

In [69]:
clf = LogisticRegression(solver = 'lbfgs',max_iter = 1000)  # initiating the classifier
#from sklearn.svm import SVC
#clf = SVC(kernel = "linear")
clf = OneVsRestClassifier(clf)

### 9. Fit the classifier, make predictions and get the accuracy (5 points)

  a. Print the following

    i. Accuracy score
    ii. F1 score
    iii. Average precision score
    iv. Average recall score

In [70]:
clf.fit(X_train,Y_train_trans) # Fitting on  train data

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 16 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 17 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 33 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 34 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 36 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 37 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 45 is present in all training 

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [71]:

print("Train Accuracy:",clf.score(X_train,Y_train_trans))

Train Accuracy: 0.9561194029850746


In [72]:
Y_pred = clf.predict(X_test)

In [73]:
print("Test Accuracy:" + str(accuracy_score(Y_test_trans, Y_pred)))
print("F1: " + str(f1_score(Y_test_trans, Y_pred, average='micro')))
print("F1_macro: " + str(f1_score(Y_test_trans, Y_pred, average='macro')))
print("Precision: " + str(precision_score(Y_test_trans, Y_pred, average='micro')))
print("Precision_macro: " + str(precision_score(Y_test_trans, Y_pred, average='macro')))
print("Recall: " + str(recall_score(Y_test_trans, Y_pred, average='micro')))
print("Recall_macro: " + str(recall_score(Y_test_trans, Y_pred, average='macro')))

Test Accuracy:0.5315151515151515
F1: 0.7432240847302143
F1_macro: 0.25418706963707105
Precision: 0.8188166115398751
Precision_macro: 0.42299519494609034
Recall: 0.6804092227821041
Recall_macro: 0.2056911276465752


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 10. Print true label and predicted label for any five examples (7.5 points)


In [74]:
Y_pred_inv = mlb.inverse_transform(Y_pred)   # inverse transforming predited label data
Y_test_trans_inv =  mlb.inverse_transform(Y_test_trans) # inverse transforming original test label data

In [84]:
print("Example 1 - predicted :",Y_pred_inv[10])
print("Example 1 - Actual :",Y_test_trans_inv[10])
print("Example 1 - Actual_before mlb transformation :",Y_test[10])

Example 1 - predicted : ('male', 'aries', '35', 'technology')
Example 1 - Actual : ('male', 'aries', '35', 'technology')
Example 1 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


In [85]:
print("Example 2 - predicted :",Y_pred_inv[320])
print("Example 2 - Actual :",Y_test_trans_inv[320])
print("Example 2 - Actual_before mlb transformation :",Y_test[320])

Example 2 - predicted : ('female', 'indunk', 'sagittarius', '34')
Example 2 - Actual : ('female', 'indunk', 'sagittarius', '34')
Example 2 - Actual_before mlb transformation : ['female', '34', 'indunk', 'sagittarius']


In [86]:
print("Example 3 - predicted :",Y_pred_inv[839])
print("Example 3 - Actual :",Y_test_trans_inv[839])
print("Example 3 - Actual_before mlb transformation :",Y_test[839])

Example 3 - predicted : ('male', 'aries', '35', 'technology')
Example 3 - Actual : ('male', 'aries', '35', 'technology')
Example 3 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


In [87]:
print("Example 4 - predicted :",Y_pred_inv[300])
print("Example 4 - Actual :",Y_test_trans_inv[300])
print("Example 4 - Actual_before mlb transformation :",Y_test[300])

Example 4 - predicted : ('male', 'leo', 'indunk', '26')
Example 4 - Actual : ('male', 'leo', 'indunk', '26')
Example 4 - Actual_before mlb transformation : ['male', '26', 'indunk', 'leo']


In [88]:
print("Example 5 - predicted :",Y_pred_inv[1289])
print("Example 5 - Actual :",Y_test_trans_inv[1289])
print("Example 5 - Actual_before mlb transformation :",Y_test[1289])

Example 5 - predicted : ('male', 'aries', '35', 'technology')
Example 5 - Actual : ('male', 'libra', '39')
Example 5 - Actual_before mlb transformation : ['male', '39', 'communicationsmedia', 'libra']
